In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time # 사이트 기동될 때 까지 2초 정도를 줌. - 셀리니움에서 get. request에서 get쓸 때
import pymysql

In [2]:
# 사이트 기동
def dongaUrl(search):
        url_temp = 'https://www.hankookilbo.com/Search?&q={search}&c=&s=1&from=2019.03.01&until=2020-05-20'.format(search=search)
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.get(url_temp)
        time.sleep(2)
        
        return dongaUrlList(driver)

In [17]:
# 2단계 목록에서 href 가져오기
def dongaUrlList(driver):
    url_temp = driver.current_url 
    dongaUrls_List = []
    for pageNo in range(1, 2, 1):
        print(pageNo)
        url = url_temp + "&Page={pageNo}".format(pageNo=pageNo)
        driver.get(url)
    
        dongaUrls = driver.find_elements_by_css_selector(".article-list .item .body a")
        for urlList in dongaUrls:
            dongaUrls_List.append(urlList.get_attribute("href"))
    return dongaInfo(driver, dongaUrls_List)

In [32]:
def dongaInfo(driver, dongaUrls_List):
    for urllist in dongaUrls_List:
        driver.get(urllist)
        time.sleep(2)
#         . 은 class, # 은 id값, HTML tag는 그대로 표현됨
        print(urllist)
    
        # 제목
        newsTitleList = driver.find_elements_by_css_selector(".article-header h3")
        newsTitle_temp = newsTitleList[0]
        newsTitle = newsTitle_temp.text
        print(newsTitle)
        
        #  소제목
#         newsSubtitle = driver.find_element_by_css_selector(".article-story .read").text
        newsSubtitle = ""
#         newsSubtitleList = driver.find_elements_by_css_selector(".article-story .read")
#         newsSubtitle_temp = newsSubtitleList[0]
#         newSubtitle = newsSubtitle_temp.text
        print(newsSubtitle)
        
        # 날짜
        newsDate = ""
        newsDateList = driver.find_elements_by_css_selector(".info")
        newsDate_temp = newsDateList[0]
        newsDate = newsDate_temp.text
        newsDate = newsDate.replace("입력 ", "").replace(".", "")[0:8]
        print(newsDate)
        
#         # 내용
#         newsContents = ""
#         newsContentsList = driver.find_elements_by_css_selector(".article-story .read")
#         for newsContent in newsContentsList:
#             newsContents = newsContents + newsContent.text
#         print(newsContents)

#         dbData = [[urllist, newsDate, newsTitle, newsSubtitle, newsContents]]
#         connectDB(dbData)

In [31]:
# search = input("검색: ")
search = '일본 불매운동'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
dongaUrl(search)

1
https://www.hankookilbo.com/News/Npath/202003261503731422
[시시콜콜Why] ‘모동숲’이 뭐지…코로나19·일본 불매운동에도 없어서 못 사는 이유


IndexError: list index out of range

In [ ]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'python'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into donga(url, newsDate, newsTitle,newsSubtitle,contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()